## Words as numbers 

Text can be hard to use in a machine learning problem. Machine learning is based on mathematics and statistics, which we all know is numbers. Text is ... well, not numbers. 

In order to make use of text, and extract information out of it we would have to "make" numbers out of it. we do that 

<img src='word2vec.png' style='width:800px'/>

In [1]:
!pip install -U tensorflow==1.14.0 gensim==3.8.0 fasttext==0.9.1 bert-serving-server==1.9.6 bert-serving-client==1.9.6

     |████████████████████████████████| 109.3MB 538kB/s  eta 0:00:01    |███████████████████████▍        | 79.9MB 16.0MB/s eta 0:00:02     |███████████████████████████▎    | 93.3MB 16.0MB/s eta 0:00:01
     |████████████████████████████████| 24.2MB 27.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 7.9MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 4.7MB/s  eta 0:00:01
     |████████████████████████████████| 491kB 21.0MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 22.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 25.8MB/s eta 0:00:01
     |████████████████████████████████| 3.2MB 16.6MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 13.9MB/s eta 0:00:01
     |████████████████████████████████| 51kB 11.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 21.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/05/4b/55cfbfd3e5e85016eeef9f21c0ec809d978706a0d60b62cc28aeec8c792f/numpy-1.17

  Using cached https://files.pythonhosted.org/packages/69/1b/b853c7a9d4f6a6d00749e94eb6f3a041e342a885b87340b79c1ef73e3a78/certifi-2019.6.16-py2.py3-none-any.whl
     |████████████████████████████████| 71kB 6.2MB/s  eta 0:00:01
     |████████████████████████████████| 5.7MB 16.0MB/s eta 0:00:01
     |████████████████████████████████| 552kB 27.4MB/s eta 0:00:01
  Created wheel for fasttext: filename=fasttext-0.9.1-cp37-cp37m-linux_x86_64.whl size=2474776 sha256=81799bd1e730f61d59aa6ad958e7ebc3a482f3d848af86b2dfc223422aa05ca6
  Stored in directory: /home/gurra/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
  Created wheel for smart-open: filename=smart_open-1.8.4-cp37-none-any.whl size=68201 sha256=6f07f95b8928a3bd81b6c2812458f9365ed4692d96042ef19e7f6a4946de1d49
  Stored in directory: /home/gurra/.cache/pip/wheels/5f/ea/fb/5b1a947b369724063b2617011f1540c44eb00e28c3d2ca8692
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7414 sha256=b9

In [9]:
def cosine_sim(s1, s2):
    return np.dot(s1,s2)/(np.linalg.norm(s1)*np.linalg.norm(s2)+1e-9)

  




In [32]:
string1 = 'The man is robbing a bank'
string2 = 'this guy is stealing money from a finance institution'

### fastText

In [4]:
%%bash 
wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip'
mkdir fasttext 
unzip 'wiki-news-300d-1M.vec.zip' -d fasttext
rm 'wiki-news-300d-1M.vec.zip'

Archive:  wiki-news-300d-1M.vec.zip
  inflating: fasttext/wiki-news-300d-1M.vec  


--2019-08-21 16:51:03--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

     0K .......... .......... .......... .......... ..........  0%  166K 66m51s
    50K .......... .......... .......... .......... ..........  0% 53,5M 33m32s
   100K .......... .......... .......... .......... ..........  0%  227K 38m39s
   150K .......... .......... .......... .......... ..........  0% 14,3M 29m10s
   200K .......... .......... .......... .......... ..........  0%  231K 32m56s
   250K .......... .......... .......... .......... ..........  0% 15,9M 27m33s
   300K .......... .......... .......... .......... ..........  0% 84,

In [7]:
def get_fasttext_wv(word, model, vec_len=300):
    try:
        wv = model[word]
    except:
        wv = np.zeros(vec_len)
    return wv


def fasttext_doc2vec(string, model, stop_words=list(), weights=dict(), smoothing_factor=1, n_dim=300):
    """
    sentence embedding from a word embedding model and string sentence. 
    args:
    string(str): the string to be embedded
    model(gensim.models.keyedvectors.Word2VecKeyedVectors): the model
    stop_words(list): optional stop words to be removed
    weights(dict): optional dict with weights of words 
    smoothing_factor(int): helps smoothing the doc2 vec. A nonzero value is as 
    if all words without an explicit weight would have that value as explict. A smoothing factor=0 
    is equivalent to ignoring words that has no explcit weight. Set to 1 if unsure.
    
    """
    words = set(re.findall("[\w\d'-]+", string.lower())) # ignores multiple uses of a word in the doc
    
    word_weights = []
    if words:
        word_vectors = [get_fasttext_wv(word, model) for word in words if word not in stop_words]
        for word in words:
            try:
                word_weights.append(weights[word])
            except:
                word_weights.append(smoothing_factor)
                
        se = np.mean([vec / (np.linalg.norm(vec) + 1e-9) * weight for vec, weight in zip(word_vectors, word_weights)], axis = 0)
    else:
        print('Zero doc2vec vector for: ' + string)
        se = np.zeros(n_dim)
    return se  



In [6]:
from gensim.models import KeyedVectors
import re
import numpy as np
filename = 'fasttext/wiki-news-300d-1M.vec'
fasttext_model = KeyedVectors.load_word2vec_format(filename)



In [41]:


str_1_emb = fasttext_doc2vec(string1, fasttext_model)
str_2_emb = fasttext_doc2vec(string2, fasttext_model)

s1_s2_sim_fasttext = cosine_sim(str_1_emb, str_2_emb)

print('Document similarity between \n\n(1){s1}\n(2){s2}\nis: {sim}'.format(s1=string1,
                                                                       s2=string2,
                                                                       sim=s1_s2_sim_fasttext))

Document similarity between 

(1)The man is robbing a bank
(2)this guy is stealing money from a finance institution
is: 0.9037829948571913


### BERT

In [16]:
%%bash 
wget 'https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip'
mkdir bert 
unzip 'uncased_L-12_H-768_A-12.zip' -d bert
rm 'uncased_L-12_H-768_A-12.zip'



Archive:  uncased_L-12_H-768_A-12.zip
  inflating: bert/ed_L-12_H-768_A-12.zip'  


--2019-08-21 17:08:42--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.211.16, 2a00:1450:400f:80d::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.211.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

     0K .......... .......... .......... .......... ..........  0% 6,70M 58s
    50K .......... .......... .......... .......... ..........  0% 9,45M 50s
   100K .......... .......... .......... .......... ..........  0% 7,37M 51s
   150K .......... .......... .......... .......... ..........  0% 12,9M 45s
   200K .......... .......... .......... .......... ..........  0% 13,5M 42s
   250K .......... .......... .......... .......... ..........  0% 7,23M 44s
   300K .......... .......... .......... .......... ..........  0% 14,2M 42s
   350K .......... .......

CalledProcessError: Command 'b"wget 'https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip'\nmkdir bert \nunzip 'uncased_L-12_H-768_A-12.zip' -d bert\nrm 'uncased_L-12_H-768_A-12.zip'\n\n"' returned non-zero exit status 1.

In [18]:
#bert-serving-start -model_dir '/home/gurra/coding/bert/bert_tutorial/bert/uncased_L-12_H-768_A-12' -num_worker=1 

Process is terminated.


In [19]:
from bert_serving.client import BertClient
import numpy as np


In [20]:
bc=BertClient()

In [40]:


s1 = bc.encode([string1])
s2 = bc.encode([string2])
s1_s2_sim_bert = cosine_sim(s1[0],s2[0])


print('Document similarity between \n\n(1){s1}\n(2){s2}\nis: {sim}'.format(s1=string1,
                                                                       s2=string2,
                                                                       sim=s1_s2_sim_bert))

Document similarity between 

(1)The man is robbing a bank
(2)this guy is stealing money from a finance institution
is: 0.8606592470371924


['The', 'man', 'is', 'robbing', 'a', 'bank']

0.8606592470371924

https://mlexplained.com/2019/06/30/paper-dissected-xlnet-generalized-autoregressive-pretraining-for-language-understanding-explained/
    